# 总线类设备驱动

> 设计总线类设备驱动的目的 

设计总线类设备驱动的目的是实现设备和驱动代码的分离，使设备驱动代码和设备驱动代码之间相互独立

设备专门用来描述设备所占有的硬件资源

驱动和设备通过总线绑定后

驱动负责从设备中动态获取这些资源信息
当设备的资源改变后，只是设备改变而已，驱动不用作过多修改，驱动代码的通用性大大提高

> USB总线设备的工作流程实例

在一台拥有USB总线的计算机上，USB总线会在外部留出许多USB接口，可以挂载很多USB设备。为了能让这些设备正常工作，系统上会安装其对应的驱动，这些驱动在软件层面上挂载在USB总线上。当接入一个USB设备时，USB总线会立刻感知到，并去遍历所有在USB总线上注册的驱动，找到匹配的驱动后会自动加载一个匹配的USB驱动，然后调用驱动中的一段代码来探测是否能驱动刚刚插入的USB设备，如果探测成功，则驱动会根据探测到的信息来初始化这个设备。

> liunx设备模型

1. struct bus_type 代表总线对象
2. struct device 代表设备对象
3. struct deivce_driver 代表驱动对象

## 总线类设备的代码编写的大体流程为：

> 总线代码

1. 先定义一个总线对象

```c
struct bus_type mybus =   
{
	.name = "mybus",	//总线的名字
	.match = mybus_match,  //匹配函数
};
```
总线结构体的类型为struct bus_type，其详细的定义在include/linux/device.h中

2. 注册上面定义的总线对象
   在mybus_init函数中调用bus_register

3. 定义一个匹配函数mybus_match
   在mybus_init中调用bus_register时，会调用mybus_match

4. 模块卸载时，调用bus_unregister

> 驱动代码

1. 定义一个驱动对象
```c
struct device_driver mydrv = 
{
    .name  = "fsdev_drv",
    .bus   = &mybus,          //驱动结构体与总线结构体关联
    .probe = mydrv_probe,     //驱动匹配函数
    .remove= mydrv_remove,    //驱动卸载函数
};
```

2. 注册驱动对象
   在mydrv_init中调用driver_register

3. 定义一个匹配函数mydrv_probe
   在mydrv_init中调用driver_register时，会调用mydrv_probe

4. 定义一个卸载函数mydrv_remove
   在mydrv_exit中调用driver_unregister，会调用mydrv_remove

5. 卸载驱动对象
   在mydrv_exit中调用driver_unregister

> 设备代码

1. 定义描述设备信息的结构体，可以在头文件中定义好，在设备代码中实例化
```c
struct mydev_desc dev_infos = 
{  // 设备的信息
    .name  = "dev_test", // 设备的名字
    .irqno = 666,      // 设备的中断号
    .addr  = 0x20033000,  // 设备的地址
};
```

2. 定义一个设备结构体
```c
struct device mydev = {
    .init_name     = "fsdev_drv",  /* initial name of the device */
    .bus           = &mybus,        //设备结构体与总线结构体关联
    .release       = mydev_release,  //设备卸载函数
    .platform_data = &dev_infos,   //自定义数据
};
```

3. 注册设备结构体
   在mydev_init中调用device_register

4. 定义一个卸载函数mydev_release
   当设备卸载时，会调用mydev_release

5. 卸载设备结构体
   在mydev_exit中调用device_unregister

In [ ]:
//总线部分vbus.c
#include <linux/init.h>
#include <linux/kernel.h>
#include <linux/module.h>

#include <linux/device.h>

static int vbus_match(struct device *dev, struct device_driver *drv)  
// 匹配函数,负责匹配设备与驱动，如果匹配成功，则返回1
{
	return 1;
}
i
static struct bus_type vbus = 
{  
	// 定义总线数据结构
	.name = "vbus",  //总线的名字是vbus
	.match = vbus_match,  //匹配函数
};

EXPORT_SYMBOL(vbus);

static int __init vbus_init(void)
{
	return bus_register(&vbus);  // 注册总线
}
static void __exit vbus_exit(void)
{
	bus_unregister(&vbus);
}
module_init(vbus_init);
module_exit(vbus_exit);

MODULE_LICENSE("GPL");
MODULE_AUTHOR("Kevin Jiang <jiangxg@farsight.com.cn>");
MODULE_DESCRIPTION("A virtual bus");

In [ ]:
//驱动部分vdrv.c
//一旦在总线中完成了匹配，驱动就动态地从设备部分中获取设备信息，然后调用驱动的probe函数
#include <linux/init.h>
#include <linux/kernel.h>
#include <linux/module.h>

#include <linux/device.h>

extern struct bus_type vbus;

static struct device_driver vdrv = {
	.name = "vdrv",
	.bus = &vbus,
};

static int __init vdrv_init(void)
{
	return driver_register(&vdrv);
}

static void __exit vdrv_exit(void)
{
	driver_unregister(&vdrv);
}

module_init(vdrv_init);
module_exit(vdrv_exit);

MODULE_LICENSE("GPL");
MODULE_AUTHOR("Kevin Jiang <jiangxg@farsight.com.cn>");
MODULE_DESCRIPTION("A virtual device driver");

In [ ]:
//设备部分vdev.c
// 保存设备所占用的资源信息
#include <linux/init.h>
#include <linux/kernel.h>
#include <linux/module.h>

#include <linux/device.h>

extern struct bus_type vbus;

static void vdev_release(struct device *dev)
{
}

static struct device vdev =   // 设备结构体
{
	.init_name = "vdev",   // 设备名称
	.bus = &vbus,           // 设备所属总线
	.release = vdev_release,  // 设备释放函数
};

static int __init vdev_init(void)
{
	return device_register(&vdev);  // 注册设备
}

static void __exit vdev_exit(void)
{
	device_unregister(&vdev);
}

module_init(vdev_init);
module_exit(vdev_exit);

MODULE_LICENSE("GPL");
MODULE_AUTHOR("Kevin Jiang <jiangxg@farsight.com.cn>");
MODULE_DESCRIPTION("A virtual device");

In [ ]:
//mybus.c
#include <linux/init.h>
#include <linux/module.h>
#include <linux/device.h>

int mybus_match(struct device *dev, struct device_driver *drv)
{
	//如果匹配成功，match方法一定要返回一个1， 失败返回0
	//因为是自己定义的总线，这里可以定义一个手动名称匹配的办法
	//先取出dev与drv的name
   	//不能直接使用dev->init_name，因为会把init_name赋给父类kobject,然后置空
	if(!strncasecmp(drv->name, dev->kobj.name, strlen(drv->name)))
	{
		printk("match ok\n");
		return 1;
	}
	else
	{
		printk("match failed\n");
		return 0;
	}    	
}

//实例化一个bus对象,用来管理device和driver，完成匹配
struct bus_type mybus = 
{
	.name = "mybus",	//总线的名字
	.match = mybus_match,  //匹配函数
};


EXPORT_SYMBOL(mybus);  //导出mybus,以便其他模块使用

static int __init mybus_init(void)
{
    printk("----------%s-------------\n", __FUNCTION__);

	//构建一个总线
	// /sys/bus/mybus
	int ret = bus_register(&mybus);  //注册总线
	if(ret != 0)
	{
		printk("bus_register error\n");
		return ret;
	}

	return 0;
}


static void __exit mybus_exit(void)
{
	printk("----------%s-------------\n", __FUNCTION__);

	bus_unregister(&mybus);
}

module_init(mybus_init);
module_exit(mybus_exit);

MODULE_LICENSE("GPL");

In [ ]:
//mydrv.c
#include <linux/init.h>
#include <linux/module.h>
#include <linux/device.h>
#include <linux/io.h>
#include "dev_info.h"

struct mydev_desc *pdesc;

//probe中设备相关数据来自struct device *dev
int mydrv_probe (struct device *dev)
{
    printk("--------------%s-------------\n",__FUNCTION__);

    pdesc = (struct mydev_desc *)dev->platform_data;

    printk("name  = %s\n", pdesc->name);
    printk("irqno = %d\n", pdesc->irqno);

    //假设要执行硬件相关操作
    unsigned long *paddr = ioremap(pdesc->addr, 8);
    
    return 0;
}

int mydrv_remove (struct device *dev)
{
    printk("--------------%s-------------\n",__FUNCTION__);
    return 0;
}

extern struct bus_type mybus;


struct device_driver mydrv = {

    .name  = "fsdev_drv",
    .bus   = &mybus,
    .probe = mydrv_probe,
    .remove= mydrv_remove,

};



static int __init mydrv_init(void)
{
    printk("--------------%s-------------\n",__FUNCTION__);

    //将驱动注册到总线中
    int ret;
    ret = driver_register(&mydrv);
    if(ret < 0)
    {
        printk("driver register failed\n");
        return ret;
    }

    return 0;
}

static void __exit mydrv_exit(void)
{
    printk("-------------%s------------\n",__FUNCTION__);
    driver_unregister(&mydrv);
}



module_init(mydrv_init);
module_exit(mydrv_exit);

MODULE_LICENSE("GPL");

In [ ]:
//dev_info.h
#ifndef __DEV_INFO_H__

#define _DEV_INFO_H__


//单独设置一个自定义数据，描述设备的特性
struct mydev_desc
{
    char *name;
    int irqno;
    unsigned long addr;
};

#endif

In [ ]:
#include <linux/init.h>
#include <linux/module.h>
#include <linux/device.h>

#include "dev_info.h"

extern struct bus_type mybus;

struct mydev_desc dev_infos = {  // 设备的信息
    .name  = "dev_test", // 设备的名字
    .irqno = 666,      // 设备的中断号
    .addr  = 0x20033000,  // 设备的地址
};

void mydev_release(struct device *dev)
{
    printk("------------%s-----------\n",__FUNCTION__);
}

//构建一个device对象
struct device mydev = {
    .init_name     = "fsdev_drv",  /* initial name of the device */
    .bus           = &mybus,
    .release       = mydev_release,
    .platform_data = &dev_infos,   //自定义数据
};

static int __init mydev_init(void)
{
    printk("------------%s-----------\n",__FUNCTION__);
    int ret;
    //将device注册到总线中去
    ret = device_register(&mydev);
    if(ret < 0)
    {
        printk("device_register failed\n");
        return ret;
    }

    return 0;
}

static int __exit mydev_exit(void)
{
    device_unregister(&mydev);

}


module_init(mydev_init);
module_exit(mydev_exit);

MODULE_LICENSE("GPL");


## 总线模块挂载后，在sys文件系统中会生成相应目录

![alt text](image-3.png)

当加载了vbus模块后

+ /sys/bus目录下会自动创建/sys/bus/vbus模块
+ /sys/bus/vbus目录下会自动创建/sys/bus/vbus/device目录
+ /sys/bus/vbus目录下会自动创建/sys/bus/vbus/driver目录

当加载了vdrv模块后

+ /sys/bus/vbus/drivers目录下自动创建/sys/bus/vbus/drivers/vdrv目录

当加载了vdev模块后

+ /sys/bus/vbus/deivce目录下自动创建/sys/bus/vus/device/vdev目录
+ /sys/deivce目录下也自动创建/sys/deivce/vdev目录

## 总线有那几种类型
1. 平台总线plaform bus
2. IIC总线
3. SPI总线
4. USB总线
5. PCI总线

# platform 总线

有的设备没有其对应的物理总线，linux系统为了管理这些设备，开发了一种虚拟总线，即platform总线。

> platform总线中最最要的两个结构体：
>> struct platform_device
```c
struct platform_device 
{
	const char	*name;          //设备名称, 比如"i2c-dev",在平台总线的match函数中可以用于匹配
	int		id;                 //设备编号，用于区别同类型的不同平台设备
	bool		id_auto;        //是否自动分配id
	struct device	dev;        //设备
	u32		num_resources;      //设备资源个数
	struct resource	*resource;  //设备资源

	const struct platform_device_id	*id_entry;  //设备id，match函数中会首先匹配该项，不成功再试用name匹配

	/* MFD cell pointer */
	struct mfd_cell *mfd_cell;  //设备对应的mfd

	/* arch specific additions */
	struct pdev_archdata	archdata;  //设备架构信息
};
```
>> struct platform_driver
```c
struct platform_driver 
{
    

## 实例1:

In [ ]:
/*设备
把每个设备个性化的资源写在这个文件
本文件实现一个led的设备
*/
#include "linux/quota.h"
#include <linux/init.h>
#include <linux/module.h>
#include <linux/platform_device.h>


#define GPIO_REG_BASE 0x11400000
#define GPF3_CON GPIO_REG_BASE + 0x01E0
#define GPF3_SIZE  24
#define GPX1_CON  GPIO_REG_BASE + 0x0C20
#define GPX1_SIZE  24

//////////////////定义一个资源数组///////////////////
// 一个设备可以有很多的资源 
struct resource  led_res[] = {
	[0] = {
		.start = GPF3_CON,
		.end = GPF3_CON + GPF3_SIZE - 1, 
		.flags = IORESOURCE_MEM,
	},
	[1] = {
		.start = GPX1_CON,
		.end = GPX1_CON + GPX1_SIZE - 1, 
		.flags = IORESOURCE_MEM,
	},
#if 0
	//有些设备也有中断资源 ，用于说明中断资源的使用，本驱动中是没有太多意义
	[2] = {
		.start =  67,
		.end = 67, 
		.flags = IORESOURCE_IRQ,
		},		
#endif
};
///////////定义一个platform_device对象////////////////////
struct platform_device led_pdev = 
{
	.name  = "exynos4412_led", 			   // 用于做匹配  /sys/bus/platform/devices/exynos4412_led
	.id	= -1,                 			   // 一般都是直接给-1，-1表示自动分配一个id 
	.num_resources = ARRAY_SIZE(led_res),  // 资源的个数,使用宏ARRAY_SIZE来获取数组的个数
	.resource = led_res,				   // 资源数组：包括了一个设备的地址和中断
};

static int __init plat_led_pdrv_init(void)
{
	//注册一个platform_devices
	return platform_device_register(&led_pdev);
	
}    

static void __exit plat_led_pdrv_exit(void)
{
	platform_device_unregister(&led_pdev);
}

module_init(plat_led_pdrv_init);
module_exit(plat_led_pdrv_exit);
MODULE_LICENSE("GPL");

In [ ]:
/*驱动部分
把驱动实现的代码逻辑放到这个文件中
*/
#include <linux/init.h>
#include <linux/module.h>
#include <linux/slab.h>
#include <linux/fs.h>
#include <linux/device.h>
#include <linux/ioport.h>
#include <linux/platform_device.h>
#include <asm/io.h>
#include <asm/uaccess.h>


//设计一个全局的设备对象，方便操作
typedef struct _led_dev
{
	int dev_major;  //设备号

	struct class *cls;  //设备类
	struct device *dev;  //设备节点

	struct resource *res; //获取到的内存资源
	void *reg_base; //表示物理地址映射之后的虚拟地址	
}LED_DEV_T;

LED_DEV_T *samsung_led;

////////////////////// 字符设备驱动的框架 ////////////////////////////
ssize_t led_pdrv_write(struct file *filp, const char __user *buf, size_t count, loff_t *fpos)
{
	int val;
	int ret;

	//从用户空间拷贝数据到内核空间
	ret = copy_from_user(&val, buf, count);
	if(ret > 0)
	{
		printk("copy_from_user error\n");
		return -EFAULT;
	}
	if(val)
	{ //亮
		writel(readl(samsung_led->reg_base + 4) | (0x3<<4) , samsung_led->reg_base+4);
	}
	else
	{
		writel(readl(samsung_led->reg_base + 4) & ~(0x3<<4) , samsung_led->reg_base+4);
	}

	return count;
	
}

int led_pdrv_open(struct inode *inode, struct file *filp)
{
	printk("-----%s------------\n", __FUNCTION__);
	return 0;

}
int led_pdrv_close(struct inode *inode, struct file *filp)
{
	printk("-----%s------------\n", __FUNCTION__);
	return 0;
}


const struct file_operations led_fops = {
	.open = led_pdrv_open,
	.release = led_pdrv_close,
	.write = led_pdrv_write,
};
/////////////////////////////////////////////////////////////

/*
a， 注册设备号，并且注册fops--为用户提供一个设备标示，同时提供文件操作io接口
b， 创建设备节点
c， 初始化硬件
			ioremap(地址);  //地址从pdev需要获取
			readl/writle();
d，实现各种io接口： xxx_open, xxx_read, ..

*/
//是驱动真正的入口函数，当匹配成功之后被调用
int led_pdrv_probe(struct platform_device *pdev)
{
	int ret;
	printk("-----%s------------\n", __FUNCTION__);

	samsung_led = kzalloc(sizeof(LED_DEV_T), GFP_KERNEL);  //给设备对象分配内存
	if(NULL == samsung_led)
	{
		printk("kzalloc errorn\n");
		return -ENOMEM;
	}	
    //注册设备号
	samsung_led->dev_major = register_chrdev(0, "led_dev", &led_fops);
	if(samsung_led->dev_major < 0)
	{
		printk(KERN_ERR "register_chrdev error\n");
		ret = -ENODEV;
		goto err_0;
	}
	else
	{
		printk("register_chrdev ok\n");
	}
    //创建类，用于创建设备节点
	samsung_led->cls = class_create(THIS_MODULE, "led_new_cls");  //创建类，“led_new_cls”是类的名称
	//在/sys/class/目录下创建一个设备类，以便udev可以创建设备节点。
	if(IS_ERR(samsung_led->cls))
	{
		printk(KERN_ERR "class_create error\n");
		ret = PTR_ERR(samsung_led->cls);
		goto err_1;
	}
	samsung_led->dev = device_create(samsung_led->cls, NULL, MKDEV(samsung_led->dev_major, 0), NULL, "led0");  //创建设备节点
	if(IS_ERR(samsung_led->dev))
	{
		printk(KERN_ERR "device_create error\n");
		ret = PTR_ERR(samsung_led->dev);
		goto err_2;
	}

	//获取资源
	// 参数1: 从哪个pdev中获取资源,pdev就是匹配到的platform_device对象
	// 参数2:  资源类型
	// 参数3: 表示获取同种资源的第几个
	samsung_led->res = platform_get_resource(pdev, IORESOURCE_MEM, 0);  //从pdev获取第0个内存资源
	if(NULL == samsung_led->res)
	{
		printk(KERN_ERR "platform_get_resource error\n");
		return -ENOMEM;
		goto err_3;
	}
    //获取中断号
	//int irqno =  platform_get_irq(pdev, 0);
	// 等同于platform_get_resource(pdev, IORESOURCE_IRQ, 0);

	//地址映射
	samsung_led->reg_base = ioremap(samsung_led->res->start,  resource_size(samsung_led->res));
	if(samsung_led->reg_base == NULL)
	{
		printk(KERN_ERR "ioremap error\n");
		ret = -ENOMEM;
		goto err_4;
	}
	else
	{
		printk("ioremap ok\n");
	}

	//对寄存器进行配置--输出功能
	writel((readl(samsung_led->reg_base) & ~(0xff<<16))| (0x11<<16) , samsung_led->reg_base);

	return 0;
	
err_4:
	iounmap(samsung_led->reg_base);
err_3:
	device_destroy(samsung_led->cls, MKDEV(samsung_led->dev_major, 0));
err_2:
	class_destroy(samsung_led->cls);
err_1:
	unregister_chrdev(samsung_led->dev_major, "led_drv");
err_0:
	kfree(samsung_led);
	return ret;
	
}

int led_pdrv_remove(struct platform_device *pdev)
{
	
	printk("-----%s------------\n", __FUNCTION__);

	iounmap(samsung_led->reg_base);
	device_destroy(samsung_led->cls, MKDEV(samsung_led->dev_major, 0));
	class_destroy(samsung_led->cls);

	unregister_chrdev(samsung_led->dev_major, "led_drv");
		
	kfree(samsung_led);
}

const struct platform_device_id led_id_table[] = {  //匹配表，记录匹配的设备
		{"exynos4412_led", 0x4444},
		{"s5pv210_led", 0x2222},
		{"s3c2410_led", 0x3333},
		{"s3c6410_led", 0x3333},
};	

struct platform_driver led_pdrv = {
	.probe = led_pdrv_probe,		//匹配成功之后被调用的函数
	.remove = led_pdrv_remove,		//device移除的时候调用的函数
	.driver = {
			.name = "samsung_led_drv",//可以用于做匹配
			// /sys/bus/platform/drivers/samsung_led_drv
		},
	.id_table = led_id_table,  //匹配表
};  

static int __init plat_led_pdev_init(void)
{
	printk("-----%s------------\n", __FUNCTION__);

	//注册一个pdrv
	return platform_driver_register(&led_pdrv);
	
}

static void __exit plat_led_pdev_exit(void)
{
	printk("-----%s------------\n", __FUNCTION__);

	platform_driver_unregister(&led_pdrv);
	
}

module_init(plat_led_pdev_init);
module_exit(plat_led_pdev_exit);
MODULE_LICENSE("GPL");

## 实例2: 名称匹配

In [ ]:
#include <linux/module.h>
#include <linux/kernel.h>
#include <linux/platform_device.h>
 
#define GPX1CON 0x11000C20
#define GPX1DAT 0x11000C24
 
#define GPX2CON 0x11000C40
#define GPX2DAT 0x11000C44
 
#define GPF3CON 0x114001E0
#define GPF3DAT 0x114001E4

void fs4412leds_dev_release(struct device *pdev)
{
	printk("fs4412leds_dev_release is called\n");
}
 
struct resource fs4412leds_dev_res [] =
{
	[0] = {.start = GPX1CON,.end=GPX1CON + 3,.name="GPX1CON",.flags = IORESOURCE_MEM},
	[1] = {.start = GPX1DAT,.end=GPX1DAT + 3,.name="GPX1DAT",.flags = IORESOURCE_MEM},
 
	[2] = {.start = GPX2CON,.end=GPX2CON + 3,.name="GPX2CON",.flags = IORESOURCE_MEM},
	[3] = {.start = GPX2DAT,.end=GPX2DAT + 3,.name="GPX2DAT",.flags = IORESOURCE_MEM},
	
	[4] = {.start = GPF3CON,.end=GPF3CON + 3,.name="GPF3CON",.flags = IORESOURCE_MEM},
	[5] = {.start = GPF3DAT,.end=GPF3DAT + 3,.name="GPF3DAT",.flags = IORESOURCE_MEM},
};
 
struct platform_device fs4412leds_device = 
{
	.name = "fs4412leds", 									//用于匹配的设备名称
	.dev.release = fs4412leds_dev_release,               	//额外从dev对象调用的释放函数方法	
	.resource = fs4412leds_dev_res,							//资源数组	
	.num_resources = ARRAY_SIZE(fs4412leds_dev_res),       	//资源数组大小
};
 
int __init fs4412leds_device_init(void)
{
	platform_device_register(&fs4412leds_device);
	return 0;
}
 
void __exit fs4412leds_device_exit(void)
{
	platform_device_unregister(&fs4412leds_device);
}
 
MODULE_LICENSE("GPL");
module_init(fs4412leds_device_init);
module_exit(fs4412leds_device_exit);

In [ ]:
#ifndef LED_DRIVER_H
#define LED_DRIVER_H
 
#define LED_DEV_MAGIC 'g'
 
#define MY_LED_OFF _IO(LED_DEV_MAGIC,0)
#define MY_LED_ON _IO(LED_DEV_MAGIC,1)
 
 
 
#endif

## 实例3: id匹配

In [ ]:
/*

    注意事项：

    1.  device模块中，id的name成员必须与struct platform_device中的name成员内容一致，因此device模块中，struct platform_device中的name成员必须指定

    2.  driver模块中，struct platform_driver成员driver的name成员必须指定，但与device模块中name可以不相同

*/
#include <linux/module.h>
#include <linux/kernel.h>
#include <linux/platform_device.h>
 
#define GPX1CON 0x11000C20
#define GPX1DAT 0x11000C24
 
#define GPX2CON 0x11000C40
#define GPX2DAT 0x11000C44
 
#define GPF3CON 0x114001E0
#define GPF3DAT 0x114001E4

void fs4412leds_dev_release(struct device *pdev)
{
	printk("fs4412leds_dev_release is called\n");
}
 
struct resource fs4412leds_dev_res [] =
{
	[0] = {.start = GPX1CON,.end=GPX1CON + 3,.name="GPX1CON",.flags = IORESOURCE_MEM},
	[1] = {.start = GPX1DAT,.end=GPX1DAT + 3,.name="GPX1DAT",.flags = IORESOURCE_MEM},
 
	[2] = {.start = GPX2CON,.end=GPX2CON + 3,.name="GPX2CON",.flags = IORESOURCE_MEM},
	[3] = {.start = GPX2DAT,.end=GPX2DAT + 3,.name="GPX2DAT",.flags = IORESOURCE_MEM},
	
	[4] = {.start = GPF3CON,.end=GPF3CON + 3,.name="GPF3CON",.flags = IORESOURCE_MEM},
	[5] = {.start = GPF3DAT,.end=GPF3DAT + 3,.name="GPF3DAT",.flags = IORESOURCE_MEM},
};
 
struct platform_device_id led_id = {  //定义一个platform_device_id结构体变量,用于进行id匹配
	.name = "fs4412leds",  //platform_device的name属性，与platform_driver的name属性要一致
};
// id_table匹配方法,通过比较platform_device的name属性和platform_driver的id_table中的name进行匹配。
struct platform_device fs4412leds_device = 
{
	.name = "fs4412leds",
	.dev.release = fs4412leds_dev_release,
	.resource = fs4412leds_dev_res,
	.num_resources = ARRAY_SIZE(fs4412leds_dev_res),
	.id_entry = &led_id,
};
 
int __init fs4412leds_device_init(void)
{
	platform_device_register(&fs4412leds_device);
	return 0;
}
 
void __exit fs4412leds_device_exit(void)
{
	platform_device_unregister(&fs4412leds_device);
}
 
MODULE_LICENSE("GPL");
module_init(fs4412leds_device_init);
module_exit(fs4412leds_device_exit);

In [ ]:
#ifndef LED_DRIVER_H
#define LED_DRIVER_H
 
#define LED_DEV_MAGIC 'g'
 
#define MY_LED_OFF _IO(LED_DEV_MAGIC,0)
#define MY_LED_ON _IO(LED_DEV_MAGIC,1)
 
 
 
#endif

In [ ]:

#include <linux/module.h>
#include <linux/kernel.h>
#include <linux/platform_device.h>
#include <linux/fs.h>
#include <linux/cdev.h>
#include <linux/wait.h>
#include <linux/sched.h>
#include <linux/poll.h>
#include <linux/slab.h>
#include <linux/mm.h>
#include <linux/io.h>
#include <asm/uaccess.h>
#include <asm/atomic.h>
 
#include "leddrv.h"
 
 
 
int major = 11;
int minor = 0;
int myled_num  = 1;
 
struct myled_dev
{
	struct cdev mydev;
 
	volatile unsigned long *pled2_con;
	volatile unsigned long *pled2_dat;
	
	volatile unsigned long *pled3_con;
	volatile unsigned long *pled3_dat;
 
	volatile unsigned long *pled4_con;
	volatile unsigned long *pled4_dat;
 
	volatile unsigned long *pled5_con;
	volatile unsigned long *pled5_dat;
};
 
struct myled_dev *pgmydev = NULL;
 
 
int myled_open(struct inode *pnode,struct file *pfile)
{
	pfile->private_data =(void *) (container_of(pnode->i_cdev,struct myled_dev,mydev));
	
	return 0;
}
 
int myled_close(struct inode *pnode,struct file *pfile)
{
	return 0;
}
 
void led_on(struct myled_dev *pmydev,int ledno)
{
	switch(ledno)
	{
		case 2:
			writel(readl(pmydev->pled2_dat) | (0x1 << 7),pmydev->pled2_dat);
			break;
		case 3:
			writel(readl(pmydev->pled3_dat) | (0x1),pmydev->pled3_dat);
			break;
		case 4:
			writel(readl(pmydev->pled4_dat) | (0x1 << 4),pmydev->pled4_dat);
			break;
		case 5:
			writel(readl(pmydev->pled5_dat) | (0x1 << 5),pmydev->pled5_dat);
			break;
	}
}
 
void led_off(struct myled_dev *pmydev,int ledno)
{
	switch(ledno)
	{
		case 2:
			writel(readl(pmydev->pled2_dat) & (~(0x1 << 7)),pmydev->pled2_dat);
			break;
		case 3:
			writel(readl(pmydev->pled3_dat) & (~(0x1)),pmydev->pled3_dat);
			break;
		case 4:
			writel(readl(pmydev->pled4_dat) & (~(0x1 << 4)),pmydev->pled4_dat);
			break;
		case 5:
			writel(readl(pmydev->pled5_dat) & (~(0x1 << 5)),pmydev->pled5_dat);
			break;
	}
}
 
 
long myled_ioctl(struct file *pfile,unsigned int cmd,unsigned long arg)
{
	struct myled_dev *pmydev = (struct myled_dev *)pfile->private_data;
 
	if(arg < 2 || arg > 5)
	{
		return -1;
	}
	switch(cmd)
	{
		case MY_LED_ON:
			led_on(pmydev,arg);
			break;
		case MY_LED_OFF:
			led_off(pmydev,arg);
			break;
		default:
			return -1;
	}
 
	return 0;
}
 
struct file_operations myops = {
	.owner = THIS_MODULE,
	.open = myled_open,
	.release = myled_close,
	.unlocked_ioctl = myled_ioctl,
};
 
void ioremap_ledreg(struct myled_dev *pmydev,struct platform_device *p_pltdev)
{
	struct resource *pres = NULL;
 
 
	pres = platform_get_resource(p_pltdev,IORESOURCE_MEM,2);
	pmydev->pled2_con = ioremap(pres->start,4);
 
	pres = platform_get_resource(p_pltdev,IORESOURCE_MEM,3);
	pmydev->pled2_dat = ioremap(pres->start,4);
 
 
	pres = platform_get_resource(p_pltdev,IORESOURCE_MEM,0);
	pmydev->pled3_con = ioremap(pres->start,4);
 
	pres = platform_get_resource(p_pltdev,IORESOURCE_MEM,1);
	pmydev->pled3_dat = ioremap(pres->start,4);
	
 
	pres = platform_get_resource(p_pltdev,IORESOURCE_MEM,4);
	pmydev->pled4_con = ioremap(pres->start,4);
 
	pres = platform_get_resource(p_pltdev,IORESOURCE_MEM,5);
	pmydev->pled4_dat = ioremap(pres->start,4);
	
	pmydev->pled5_con = pmydev->pled4_con;
	pmydev->pled5_dat = pmydev->pled4_dat;
}
 
void set_output_ledconreg(struct myled_dev *pmydev)
{
	writel((readl(pmydev->pled2_con) & (~(0xF << 28))) | (0x1 << 28),pmydev->pled2_con);
	writel((readl(pmydev->pled3_con) & (~(0xF))) | (0x1),pmydev->pled3_con);
	writel((readl(pmydev->pled4_con) & (~(0xF << 16))) | (0x1 << 16),pmydev->pled4_con);
	writel((readl(pmydev->pled5_con) & (~(0xF << 20))) | (0x1 << 20),pmydev->pled5_con);
 
	writel(readl(pmydev->pled2_dat) & (~(0x1 << 7)),pmydev->pled2_dat);
	writel(readl(pmydev->pled3_dat) & (~(0x1)),pmydev->pled3_dat);
	writel(readl(pmydev->pled4_dat) & (~(0x1 << 4)),pmydev->pled4_dat);
	writel(readl(pmydev->pled5_dat) & (~(0x1 << 5)),pmydev->pled5_dat);
}
 
void iounmap_ledreg(struct myled_dev *pmydev)
{
	iounmap(pmydev->pled2_con);
	pmydev->pled2_con = NULL;
	iounmap(pmydev->pled2_dat);
	pmydev->pled2_dat = NULL;
 
	iounmap(pmydev->pled3_con);
	pmydev->pled3_con = NULL;
	iounmap(pmydev->pled3_dat);
	pmydev->pled3_dat = NULL;
	
	iounmap(pmydev->pled4_con);
	pmydev->pled4_con = NULL;
	iounmap(pmydev->pled4_dat);
	pmydev->pled4_dat = NULL;
	
	pmydev->pled5_con = NULL;
	pmydev->pled5_dat = NULL;
}
 
int fs4412leds_driver_probe(struct platform_device *p_pltdev)
{
	int ret = 0;
	dev_t devno = MKDEV(major,minor);
 
	/*申请设备号*/
	ret = register_chrdev_region(devno,myled_num,"myled");
	if(ret)
	{
		ret = alloc_chrdev_region(&devno,minor,myled_num,"myled");
		if(ret)
		{
			printk("get devno failed\n");
			return -1;
		}
		major = MAJOR(devno);//容易遗漏，注意
	}
 
	pgmydev = (struct myled_dev *)kmalloc(sizeof(struct myled_dev),GFP_KERNEL);
	if(NULL == pgmydev)
	{
		unregister_chrdev_region(devno,myled_num);
		printk("kmalloc failed\n");
		return -1;
	}
	memset(pgmydev,0,sizeof(struct myled_dev));
 
	/*给struct cdev对象指定操作函数集*/	
	cdev_init(&pgmydev->mydev,&myops);
 
	/*将struct cdev对象添加到内核对应的数据结构里*/
	pgmydev->mydev.owner = THIS_MODULE;
	cdev_add(&pgmydev->mydev,devno,myled_num);
 
	/*ioremap*/
	ioremap_ledreg(pgmydev,p_pltdev);
 
	/*con-register set output*/
	set_output_ledconreg(pgmydev);
 
	return 0;
}
 
int fs4412leds_driver_remove(struct platform_device *p_pltdev)
{
	dev_t devno = MKDEV(major,minor);
 
	/*iounmap*/
	iounmap_ledreg(pgmydev);
 
	cdev_del(&pgmydev->mydev);
 
	unregister_chrdev_region(devno,myled_num);
 
	kfree(pgmydev);
	pgmydev = NULL;
 
	return 0;
}
 
struct platform_device_id leddrv_ids[] =
{
	[0] = {.name = "fs4412leds"},
	[1] = {.name = "xyz"},
	[2] = {}
};
 
struct platform_driver fs4412leds_driver = 
{
	.driver.name = "leds",
	.probe = fs4412leds_driver_probe,
	.remove = fs4412leds_driver_remove,
	.id_table = leddrv_ids,
};
 
int __init fs4412leds_driver_init(void)
{
	platform_driver_register(&fs4412leds_driver);
	return 0;
}
 
void __exit fs4412leds_driver_exit(void)
{
	platform_driver_unregister(&fs4412leds_driver);
}
 
MODULE_LICENSE("GPL");
 
module_init(fs4412leds_driver_init);
module_exit(fs4412leds_driver_exit);



## 实例4: 设备树匹配

In [ ]:
#ifndef LED_DRIVER_H
#define LED_DRIVER_H
 
#define LED_DEV_MAGIC 'g'
 
#define MY_LED_OFF _IO(LED_DEV_MAGIC,0)
#define MY_LED_ON _IO(LED_DEV_MAGIC,1) 
 
#endif

In [ ]:
/*

设备树匹配：内核启动时根据设备树自动产生的设备 ------ 优先级最高

    注意事项：

    1. 无需编写device模块，只需编写driver模块

    2. 使用compatible属性进行匹配，注意设备树中compatible属性值不要包含空白字符

    3. id_table可不设置，但struct platform_driver成员driver的name成员必须设置

*/
#include <linux/module.h>
#include <linux/kernel.h>
#include <linux/platform_device.h>
#include <linux/fs.h>
#include <linux/gpio.h>
#include <linux/of_gpio.h>
#include <linux/cdev.h>
#include <linux/wait.h>
#include <linux/sched.h>
#include <linux/poll.h>
#include <linux/slab.h>
#include <linux/mm.h>
#include <linux/io.h>
#include <asm/uaccess.h>
#include <asm/atomic.h>
 
#include "leddrv.h"
 
int major = 11;
int minor = 0;
int myled_num  = 1;
// 定义一个表示led设备信息的结构体
struct myled_dev
{
	struct cdev mydev;  //继承字符设备
 
	unsigned int led2gpio;  //定义一个gpio号
	unsigned int led3gpio;
	unsigned int led4gpio;
	unsigned int led5gpio;
};
 
struct myled_dev *pgmydev = NULL;  //定义一个全局指针用于操作设备
 
int myled_open(struct inode *pnode, struct file *pfile)
{
	pfile->private_data = (void *) (container_of(pnode->i_cdev, struct myled_dev, mydev));
	
	return 0;
}
 
int myled_close(struct inode *pnode, struct file *pfile)
{
	return 0;
}
 
void led_on(struct myled_dev *pmydev, int ledno)
{
	switch(ledno)
	{
		case 2:
			gpio_set_value(pmydev->led2gpio, 1);  //设置gpio为高电平
			break;
		case 3:
			gpio_set_value(pmydev->led3gpio, 1);
			break;
		case 4:
			gpio_set_value(pmydev->led4gpio, 1);
			break;
		case 5:
			gpio_set_value(pmydev->led5gpio, 1);
			break;
	}
}
 
void led_off(struct myled_dev *pmydev,int ledno)
{
	switch(ledno)
	{
		case 2:
			gpio_set_value(pmydev->led2gpio, 0);
			break;
		case 3:
			gpio_set_value(pmydev->led3gpio, 0);
			break;
		case 4:
			gpio_set_value(pmydev->led4gpio, 0);
			break;
		case 5:
			gpio_set_value(pmydev->led5gpio, 0);
			break;
	}
}
 
 
long myled_ioctl(struct file *pfile, unsigned int cmd, unsigned long arg)
{
	struct myled_dev *pmydev = (struct myled_dev *)pfile->private_data;
 
	if(arg < 2 || arg > 5)
	{
		return -1;
	}
	switch(cmd)
	{
		case MY_LED_ON:
			led_on(pmydev, arg);
			break;
		case MY_LED_OFF:
			led_off(pmydev, arg);
			break;
		default:
			return -1;
	}
 
	return 0;
}
 
struct file_operations myops = {
	.owner = THIS_MODULE,
	.open = myled_open,
	.release = myled_close,
	.unlocked_ioctl = myled_ioctl,
};
 
void request_leds_gpio(struct myled_dev *pmydev, struct device_node *pnode)
{
	pmydev->led2gpio = of_get_named_gpio(pnode,"led2-gpio",0);  
	gpio_request(pmydev->led2gpio,"led2");
	
	pmydev->led3gpio = of_get_named_gpio(pnode,"led3-gpio",0);
	gpio_request(pmydev->led3gpio,"led3");
	
	pmydev->led4gpio = of_get_named_gpio(pnode,"led4-gpio",0);
	gpio_request(pmydev->led4gpio,"led4");
	
	pmydev->led5gpio = of_get_named_gpio(pnode,"led5-gpio",0);
	gpio_request(pmydev->led5gpio,"led5");
}
 
void set_leds_gpio_output(struct myled_dev *pmydev)
{
	gpio_direction_output(pmydev->led2gpio,0);
	gpio_direction_output(pmydev->led3gpio,0);
	gpio_direction_output(pmydev->led4gpio,0);
	gpio_direction_output(pmydev->led5gpio,0);
}
 
void free_leds_gpio(struct myled_dev *pmydev)
{
	gpio_free(pmydev->led2gpio);
	gpio_free(pmydev->led3gpio);
	gpio_free(pmydev->led4gpio);
	gpio_free(pmydev->led5gpio);
}
 
int myled_probe(struct platform_device *p_pltdev)
{
	int ret = 0;
	dev_t devno = MKDEV(major,minor);
	struct device_node *pnode = NULL;

	// 【代码逻辑块1】申请设备号
	/*申请设备号*/
	ret = register_chrdev_region(devno,myled_num,"myled");
	if(ret)
	{
		ret = alloc_chrdev_region(&devno,minor,myled_num,"myled");
		if(ret)
		{
			printk("get devno failed\n");
			return -1;
		}
		major = MAJOR(devno);//容易遗漏，注意
	}

	// 【代码逻辑块2】分配内存并初始化
	pgmydev = (struct myled_dev *)kmalloc(sizeof(struct myled_dev), GFP_KERNEL);
	if(NULL == pgmydev)
	{
		unregister_chrdev_region(devno,myled_num);
		printk("kmalloc failed\n");
		return -1;
	}
	memset(pgmydev,0,sizeof(struct myled_dev));

	// 【代码逻辑块3】设置cdev对象
	// 【代码逻辑块3.1】给struct cdev对象指定操作函数集
	/*给struct cdev对象指定操作函数集*/
	cdev_init(&pgmydev->mydev, &myops);

	// 【代码逻辑块3.2】将struct cdev对象添加到内核对应的数据结构里
	/*将struct cdev对象添加到内核对应的数据结构里*/
	pgmydev->mydev.owner = THIS_MODULE;
	cdev_add(&pgmydev->mydev, devno, myled_num);

	// 【代码逻辑块4】获取设备树节点
	pnode = p_pltdev->dev.of_node;  // 获取设备树节点

	// 【代码逻辑块5】设置GPIO并映射
	// 【代码逻辑块5.1】ioremap
	/*ioremap*/
	request_leds_gpio(pgmydev, pnode);

	// 【代码逻辑块5.2】设置GPIO为输出
	/*con-register set output*/
	set_leds_gpio_output(pgmydev);

	return 0;
}
 
int myled_remove(struct platform_device *p_pltdev)
{
	dev_t devno = MKDEV(major,minor);
 
	/*iounmap*/
	free_leds_gpio(pgmydev);
 
	cdev_del(&pgmydev->mydev);
 
	unregister_chrdev_region(devno,myled_num);
 
	kfree(pgmydev);
	pgmydev = NULL;
 
	return 0;
}
 
struct of_device_id myleddrv_of_ids[] = 
{
	[0] = {.compatible = "fs4412,led2-5"},
	[1] = {.compatible = "origen4412,led6-9"},
	[2] = {},
};
 
struct platform_driver myled_driver = 
{
	.driver = {
		.name = "fs4412leds",
		.of_match_table = myleddrv_of_ids,
	},
	.probe = myled_probe,
	.remove = myled_remove,
};
// 驱动程序的名称被设置为 “fs4412leds”，并且
//of_match_table 字段被设置为 myleddrv_of_ids，这意味着该驱动程序将用于与 myleddrv_of_ids 数组中定义的设备进行匹配
//unsigned int led2gpio; 
int __init myled_init(void)
{
	platform_driver_register(&myled_driver); 
	return 0;
}
 
void __exit myled_exit(void)
{
	platform_driver_unregister(&myled_driver);
}
 
 
MODULE_LICENSE("GPL");
 
module_init(myled_init);
module_exit(myled_exit);